The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. <br>
Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and predict the sales of each product at a particular outlet.<br>
Using this model, BigMart will try to understand the properties of products and outlets which play a key role in increasing sales.<br>
Please note that the data may have missing values as some stores might not report all the data due to technical glitches. <br>Hence, it will be required to treat them accordingly.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score,mean_squared_error
from math import sqrt

In [ ]:
df = pd.read_csv('Datasets/bigmart.csv')
df.head(10)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.Item_Fat_Content.dtype

Let us check the unique values of each categorical column

In [ ]:
for col in df.columns:
    if df[col].dtype == "object":
        print(f"The unique values for the column {col} are: \n\t\t{df[col].unique()} and the no of unique values are {df[col].nunique()}")

**Observations**<br>
<li>It can be seen the column Item_Fat_Content is having 3 different names for the category Low Fat ("Low Fat","low fat","LF") and two different names for the category Regular ("Regular","reg")</li>

In [ ]:
# Let us correct the categories in the column Item_Fat_Content 
category_mapping = {
    'Low Fat': 'Low Fat',
    'low fat': 'Low Fat',
    'LF': 'Low Fat',
    'Regular': 'Regular',
    'reg': 'Regular'
}
df['Item_Fat_Content'] = df['Item_Fat_Content'].map(category_mapping)

df["Item_Fat_Content"].unique()

Outlet_Establishment_Year can be converted into the age of the Outlet.

In [ ]:
df['Outlet_Age'] = 2024-df['Outlet_Establishment_Year']
df.head()

In [ ]:
# Dropping the Outlet_Establishment_Year column as we have create the Outlet_Age column
df.drop(columns=["Outlet_Establishment_Year"],inplace=True)
df.columns

Let us check the missing values

In [ ]:
df.isna().sum()  # Item_Weight -- 1463 and Outlet_Size -- 2410

In [ ]:
df.describe()

Let us check the value counts of each category in each of the categorical columns

In [ ]:
for col in df.columns:
    if df[col].dtype == "object":
        print(f"Value counts for the column {col} are: \n\n{df[col].value_counts()}\n")

Handling Missing Values

In [ ]:
# Let us check the KDE of Item_Weight 
sns.kdeplot(x=df.Item_Weight)

In [ ]:
{"Mean of Item Weight":df.Item_Weight.mean(), "Mode of Item Weight":df.Item_Weight.mode()[0], "Median of Item Weight":df.Item_Weight.median()}

**Observations**<br>
The mean, median and mode are approximately equal and give a true representation of the central tendency of the column Item_Weight.

**Conclusion**<br>
Therefore, we fill the missing values of the Item_Weight column with its mean

In [ ]:
df['Item_Weight'] = df['Item_Weight'].fillna(df['Item_Weight'].mean())

For the Outlet_Size column we are choosing to relace the missing values with the mode of the column that is "Medium"

In [ ]:
df['Outlet_Size'] = df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0])

In [ ]:
df.isna().sum()

### Univariate Analysis

In [ ]:
df['Item_Visibility'].hist(bins=20)
plt.title('Item Visibility')

In [ ]:
plt.boxplot(df['Item_Visibility'])

In [ ]:
# Filtering the dataset with no outliers with respect to the Item_Visibility Column
Q1 = df['Item_Visibility'].quantile(0.25)
Q3 = df['Item_Visibility'].quantile(0.75)
IQR = (Q3-Q1)
filt_df = df.query('(@Q1 - 1.5*@IQR) <= Item_Visibility <=(@Q3 + 1.5 * @IQR)')   ## Formula for outlier

In [ ]:
filt_df.head()

In [ ]:
filt_df.shape , df.shape

In [ ]:
df.shape[0] - filt_train.shape[0] # This gives the no of rows that will be removed on filtering

In [ ]:
# We will continue with the filtered data 
df = filt_train

In [ ]:
df.loc[:,'Item_Visibility_bins'] = pd.cut(df['Item_Visibility'],[0.000,0.065,0.13,0.2],labels=['Low Viz','Viz','High Viz'])

In [ ]:
df.groupby(['Item_Visibility']).max()['Item_Visibility_bins']

In [ ]:
# Checking the missing values in the Item_Visibility_bins column
df["Item_Visibility_bins"].isna().sum()

**Observations** - The Item_Visibility_bins column is having missing values for the items which have a zero visibility.

In [ ]:
# Filling the missing values with Low_Viz
df.loc[:,'Item_Visibility_bins'] = df['Item_Visibility_bins'].fillna('Low Viz')

In [ ]:
df["Item_Visibility_bins"].isnull().sum()

## Label Encoding

Now we perform Label Encoding for the categorical columns with ordinal categories

In [ ]:
le = LabelEncoder()

In [ ]:
df.loc[:,'Item_Fat_Content'] = le.fit_transform(df['Item_Fat_Content'])

In [ ]:
df.loc[:,'Item_Visibility_bins'] = le.fit_transform(df['Item_Visibility_bins'].astype(str))

In [ ]:
df.loc[:,'Outlet_Size'] = le.fit_transform(df['Outlet_Size'])

In [ ]:
df.loc[:,'Outlet_Location_Type'] = le.fit_transform(df['Outlet_Location_Type'])

### One-hot encoding

Let's create dummy variables for categorical columns with nominal categories

In [ ]:
## Create dummy variable for Outlet Type
dummy = pd.get_dummies(df['Outlet_Type'],drop_first=True)
dummy.head()

In [ ]:
df = pd.concat([df,dummy],axis=1)

In [ ]:
df

In [ ]:
df.dtypes

### Dropping unnecessary columns

In [ ]:
# Removing unnecessary column
df = df.drop(['Item_Identifier','Item_Type','Outlet_Identifier','Outlet_Type'],axis=1)
df.columns

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

# Model Training

In [ ]:
X= df.drop(columns=["Item_Outlet_Sales"])
y = df.Item_Outlet_Sales

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.3,random_state=42)
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

### Standardization 

In [ ]:
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

### Linear Regression Model

In [ ]:
linear = LinearRegression()
linear.fit(xtrain,ytrain)

In [ ]:
print(f"The intercept is {linear.intercept_}") # Gives the intercept
print(f"The coefficients are {linear.coef_}") # Gives the coefficient

In [ ]:
preds_train = linear.predict(xtrain)
preds_test = linear.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error
print(f"RMSE for train data is : {np.sqrt(mean_squared_error(ytrain,preds_train))}")
print(f"R2 for train data is {r2_score(ytrain,preds_train)}")
print(f"RMSE for test data is : {np.sqrt(mean_squared_error(ytest,preds_test))}")
print(f"R2 for test data is {r2_score(ytest,preds_test)}")


### Ridge Regression

In [ ]:
ridge = Ridge() # alpha =1 by default
ridge.fit(xtrain,ytrain)

In [ ]:
print(f"The intercept is {ridge.intercept_}") # Gives the intercept
print(f"The coefficients are {ridge.coef_}") # Gives the coefficient

In [ ]:
preds_train = ridge.predict(xtrain)
preds_test = ridge.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error
print(f"RMSE for train data is : {np.sqrt(mean_squared_error(ytrain,preds_train))}")
print(f"R2 for train data is {r2_score(ytrain,preds_train)}")
print(f"RMSE for test data is : {np.sqrt(mean_squared_error(ytest,preds_test))}")
print(f"R2 for test data is {r2_score(ytest,preds_test)}")


### Lasso Regression

In [ ]:
lasso = Lasso() # alpha =1 by default
lasso.fit(xtrain,ytrain)

In [ ]:
print(f"The intercept is {lasso.intercept_}") # Gives the intercept
print(f"The coefficients are {lasso.coef_}") # Gives the coefficient

In [ ]:
preds_train = lasso.predict(xtrain)
preds_test = lasso.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error
print(f"RMSE for train data is : {np.sqrt(mean_squared_error(ytrain,preds_train))}")
print(f"R2 for train data is {r2_score(ytrain,preds_train)}")
print(f"RMSE for test data is : {np.sqrt(mean_squared_error(ytest,preds_test))}")
print(f"R2 for test data is {r2_score(ytest,preds_test)}")


### ElasticNet

In [ ]:
elastic = ElasticNet(alpha=1) # alpha =1 by default
elastic.fit(xtrain,ytrain)

In [ ]:
print(f"The intercept is {elastic.intercept_}") # Gives the intercept
print(f"The coefficients are {elastic.coef_}") # Gives the coefficient

In [ ]:
preds_train = elastic.predict(xtrain)
preds_test = elastic.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error
print(f"RMSE for train data is : {np.sqrt(mean_squared_error(ytrain,preds_train))}")
print(f"R2 for train data is {r2_score(ytrain,preds_train)}")
print(f"RMSE for test data is : {np.sqrt(mean_squared_error(ytest,preds_test))}")
print(f"R2 for test data is {r2_score(ytest,preds_test)}")
